**optimizer=adam (best)**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# =========================================================
# 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v

# =========================================================
# 데이터 전처리 함수
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출']
        .astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df

def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    df['roll_std28'] = df['일매출'].rolling(28).std()
    return df

# =========================================================
# Sliding Window
# =========================================================
def create_sequences(df, feature_cols, seq_len=28):
    X, y = [], []
    values = df[feature_cols].values
    targets = df['일매출'].values
    for i in range(seq_len, len(df)):
        X.append(values[i-seq_len:i])
        y.append(targets[i])
    return np.array(X), np.array(y)

# =========================================================
# Dataset
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# =========================================================
# ⭐⭐ 2-Layer GRU 모델 ⭐⭐
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.gru = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])

# =========================================================
# Autoregressive 예측
# =========================================================
def autoregressive_forecast(model, df_train, df_test, seq_len, feature_cols, scaler_X, scaler_y):

    history = df_train.copy()
    preds = []

    for i in range(len(df_test)):
        row = df_test.iloc[i].copy()
        last_vals = history['일매출'].values

        # lag 업데이트
        row['lag1'] = last_vals[-1]
        row['lag7'] = last_vals[-7]
        row['lag14'] = last_vals[-14]
        row['lag28'] = last_vals[-28]

        # rolling 업데이트
        row['roll_mean7']  = pd.Series(last_vals[-7:]).mean()
        row['roll_mean14'] = pd.Series(last_vals[-14:]).mean()
        row['roll_mean28'] = pd.Series(last_vals[-28:]).mean()

        row['roll_std7']  = pd.Series(last_vals[-7:]).std()
        row['roll_std28'] = pd.Series(last_vals[-28:]).std()

        # 시퀀스 구성
        seq_df = history.tail(seq_len).copy()
        seq_scaled = scaler_X.transform(seq_df[feature_cols])
        X = torch.tensor(seq_scaled, dtype=torch.float32).unsqueeze(0)

        # 예측
        pred_scaled = model(X).item()
        pred = scaler_y.inverse_transform([[pred_scaled]])[0][0]
        pred = max(pred, 0)

        preds.append(pred)
        row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([row])], ignore_index=True)

    return preds

# =========================================================
# 🔥 메인 실행
# =========================================================
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자'] = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)

# 학사일정 merge
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')
academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend':'acad_weekend',
    'holiday':'acad_holiday',
    'semester':'acad_semester',
    'seasonal':'acad_seasonal',
    'exam':'acad_exam',
    'ceremony':'acad_ceremony',
    'dormitory':'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test = test.merge(academic, on='영업일자', how='left')

train = add_date_features(train)
test = add_date_features(test)

train = add_lag_features(train).dropna().reset_index(drop=True)
test = add_lag_features(test)

# =========================================================
# 최종 Feature 15개
# =========================================================
feature_cols = [
    'acad_weekend', 'acad_semester', 'acad_weekday',
    'open_hours', 'acad_ceremony', 'acad_exam',
    'lag1','lag7','lag14','lag28',
    'roll_std7','roll_std28',
    'roll_mean7','roll_mean14','roll_mean28'
]

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[['일매출']] = scaler_y.fit_transform(train[['일매출']])

# =========================================================
# Sequence Length = 28
# =========================================================
seq_len = 28
X_train, y_train = create_sequences(train, feature_cols, seq_len)

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# ⭐ 2-Layer GRU 모델
model = GRUModel(input_dim=len(feature_cols), num_layers=2)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습 50 epochs
for epoch in range(50):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        loss = criterion(model(Xb), yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/50] Loss: {total_loss:.4f}")

# =========================================================
# Test 예측
# =========================================================
test_pred = autoregressive_forecast(model, train, test, seq_len, feature_cols, scaler_X, scaler_y)

mae, rmse, smape_val = evaluate(test['일매출'], test_pred)

print("\n====== 최종 성능 (2-Layer GRU + 15 features + seq_len=28) ======")
print("MAE :", mae)
print("RMSE:", rmse)
print("SMAPE:", smape_val)

test['예측매출'] = test_pred
print(test[['영업일자', '예측매출']])

test[['영업일자', '예측매출']].to_csv("prediction_full_output.csv", index=False)


[Epoch 1/50] Loss: 0.8714
[Epoch 2/50] Loss: 0.5253
[Epoch 3/50] Loss: 0.4432
[Epoch 4/50] Loss: 0.3666
[Epoch 5/50] Loss: 0.3487
[Epoch 6/50] Loss: 0.3307
[Epoch 7/50] Loss: 0.3320
[Epoch 8/50] Loss: 0.3193
[Epoch 9/50] Loss: 0.3023
[Epoch 10/50] Loss: 0.2984
[Epoch 11/50] Loss: 0.2922
[Epoch 12/50] Loss: 0.2591
[Epoch 13/50] Loss: 0.2478
[Epoch 14/50] Loss: 0.2435
[Epoch 15/50] Loss: 0.2300
[Epoch 16/50] Loss: 0.2335
[Epoch 17/50] Loss: 0.2289
[Epoch 18/50] Loss: 0.2137
[Epoch 19/50] Loss: 0.2119
[Epoch 20/50] Loss: 0.2062
[Epoch 21/50] Loss: 0.2119
[Epoch 22/50] Loss: 0.2106
[Epoch 23/50] Loss: 0.2130
[Epoch 24/50] Loss: 0.2027
[Epoch 25/50] Loss: 0.2083
[Epoch 26/50] Loss: 0.2082
[Epoch 27/50] Loss: 0.2012
[Epoch 28/50] Loss: 0.1947
[Epoch 29/50] Loss: 0.2009
[Epoch 30/50] Loss: 0.1999
[Epoch 31/50] Loss: 0.2094
[Epoch 32/50] Loss: 0.2004
[Epoch 33/50] Loss: 0.1923
[Epoch 34/50] Loss: 0.1984
[Epoch 35/50] Loss: 0.1918
[Epoch 36/50] Loss: 0.2006
[Epoch 37/50] Loss: 0.1884
[Epoch 38/

**optimizer=RMSprop**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# =========================================================
# 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v

# =========================================================
# 데이터 전처리 함수
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출']
        .astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df

def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    df['roll_std28'] = df['일매출'].rolling(28).std()
    return df

# =========================================================
# Sliding Window
# =========================================================
def create_sequences(df, feature_cols, seq_len=28):
    X, y = [], []
    values = df[feature_cols].values
    targets = df['일매출'].values
    for i in range(seq_len, len(df)):
        X.append(values[i-seq_len:i])
        y.append(targets[i])
    return np.array(X), np.array(y)

# =========================================================
# Dataset
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# =========================================================
# ⭐⭐ 2-Layer GRU 모델 ⭐⭐
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.gru = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])

# =========================================================
# Autoregressive 예측
# =========================================================
def autoregressive_forecast(model, df_train, df_test, seq_len, feature_cols, scaler_X, scaler_y):

    history = df_train.copy()
    preds = []

    for i in range(len(df_test)):
        row = df_test.iloc[i].copy()
        last_vals = history['일매출'].values

        # lag 업데이트
        row['lag1'] = last_vals[-1]
        row['lag7'] = last_vals[-7]
        row['lag14'] = last_vals[-14]
        row['lag28'] = last_vals[-28]

        # rolling 업데이트
        row['roll_mean7']  = pd.Series(last_vals[-7:]).mean()
        row['roll_mean14'] = pd.Series(last_vals[-14:]).mean()
        row['roll_mean28'] = pd.Series(last_vals[-28:]).mean()

        row['roll_std7']  = pd.Series(last_vals[-7:]).std()
        row['roll_std28'] = pd.Series(last_vals[-28:]).std()

        # 시퀀스 구성
        seq_df = history.tail(seq_len).copy()
        seq_scaled = scaler_X.transform(seq_df[feature_cols])
        X = torch.tensor(seq_scaled, dtype=torch.float32).unsqueeze(0)

        # 예측
        pred_scaled = model(X).item()
        pred = scaler_y.inverse_transform([[pred_scaled]])[0][0]
        pred = max(pred, 0)

        preds.append(pred)
        row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([row])], ignore_index=True)

    return preds

# =========================================================
# 🔥 메인 실행
# =========================================================
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자'] = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)

# 학사일정 merge
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')
academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend':'acad_weekend',
    'holiday':'acad_holiday',
    'semester':'acad_semester',
    'seasonal':'acad_seasonal',
    'exam':'acad_exam',
    'ceremony':'acad_ceremony',
    'dormitory':'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test = test.merge(academic, on='영업일자', how='left')

train = add_date_features(train)
test = add_date_features(test)

train = add_lag_features(train).dropna().reset_index(drop=True)
test = add_lag_features(test)

# =========================================================
# 최종 Feature 15개
# =========================================================
feature_cols = [
    'acad_weekend', 'acad_semester', 'acad_weekday',
    'open_hours', 'acad_ceremony', 'acad_exam',
    'lag1','lag7','lag14','lag28',
    'roll_std7','roll_std28',
    'roll_mean7','roll_mean14','roll_mean28'
]

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[['일매출']] = scaler_y.fit_transform(train[['일매출']])

# =========================================================
# Sequence Length = 28
# =========================================================
seq_len = 28
X_train, y_train = create_sequences(train, feature_cols, seq_len)

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# ⭐ 2-Layer GRU 모델
model = GRUModel(input_dim=len(feature_cols), num_layers=2)

criterion = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

# 학습 50 epochs
for epoch in range(50):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        loss = criterion(model(Xb), yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/50] Loss: {total_loss:.4f}")

# =========================================================
# Test 예측
# =========================================================
test_pred = autoregressive_forecast(model, train, test, seq_len, feature_cols, scaler_X, scaler_y)

mae, rmse, smape_val = evaluate(test['일매출'], test_pred)

print("\n====== 최종 성능 (2-Layer GRU + 15 features + seq_len=28) ======")
print("MAE :", mae)
print("RMSE:", rmse)
print("SMAPE:", smape_val)

test['예측매출'] = test_pred
print(test[['영업일자', '예측매출']])


[Epoch 1/50] Loss: 1.0675
[Epoch 2/50] Loss: 0.4683
[Epoch 3/50] Loss: 0.4144
[Epoch 4/50] Loss: 0.3798
[Epoch 5/50] Loss: 0.3650
[Epoch 6/50] Loss: 0.3299
[Epoch 7/50] Loss: 0.3347
[Epoch 8/50] Loss: 0.3130
[Epoch 9/50] Loss: 0.2906
[Epoch 10/50] Loss: 0.2704
[Epoch 11/50] Loss: 0.2747
[Epoch 12/50] Loss: 0.2480
[Epoch 13/50] Loss: 0.2488
[Epoch 14/50] Loss: 0.2473
[Epoch 15/50] Loss: 0.2613
[Epoch 16/50] Loss: 0.2499
[Epoch 17/50] Loss: 0.2300
[Epoch 18/50] Loss: 0.2316
[Epoch 19/50] Loss: 0.2409
[Epoch 20/50] Loss: 0.2166
[Epoch 21/50] Loss: 0.2198
[Epoch 22/50] Loss: 0.2175
[Epoch 23/50] Loss: 0.2230
[Epoch 24/50] Loss: 0.2201
[Epoch 25/50] Loss: 0.2067
[Epoch 26/50] Loss: 0.2264
[Epoch 27/50] Loss: 0.2102
[Epoch 28/50] Loss: 0.2225
[Epoch 29/50] Loss: 0.2137
[Epoch 30/50] Loss: 0.2131
[Epoch 31/50] Loss: 0.2067
[Epoch 32/50] Loss: 0.2129
[Epoch 33/50] Loss: 0.2074
[Epoch 34/50] Loss: 0.2247
[Epoch 35/50] Loss: 0.2058
[Epoch 36/50] Loss: 0.2003
[Epoch 37/50] Loss: 0.1976
[Epoch 38/

**SGD**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# =========================================================
# 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v

# =========================================================
# 데이터 전처리 함수
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출']
        .astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df

def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    df['roll_std28'] = df['일매출'].rolling(28).std()
    return df

# =========================================================
# Sliding Window
# =========================================================
def create_sequences(df, feature_cols, seq_len=28):
    X, y = [], []
    values = df[feature_cols].values
    targets = df['일매출'].values
    for i in range(seq_len, len(df)):
        X.append(values[i-seq_len:i])
        y.append(targets[i])
    return np.array(X), np.array(y)

# =========================================================
# Dataset
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# =========================================================
# ⭐⭐ 2-Layer GRU 모델 ⭐⭐
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.gru = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])

# =========================================================
# Autoregressive 예측
# =========================================================
def autoregressive_forecast(model, df_train, df_test, seq_len, feature_cols, scaler_X, scaler_y):

    history = df_train.copy()
    preds = []

    for i in range(len(df_test)):
        row = df_test.iloc[i].copy()
        last_vals = history['일매출'].values

        # lag 업데이트
        row['lag1'] = last_vals[-1]
        row['lag7'] = last_vals[-7]
        row['lag14'] = last_vals[-14]
        row['lag28'] = last_vals[-28]

        # rolling 업데이트
        row['roll_mean7']  = pd.Series(last_vals[-7:]).mean()
        row['roll_mean14'] = pd.Series(last_vals[-14:]).mean()
        row['roll_mean28'] = pd.Series(last_vals[-28:]).mean()

        row['roll_std7']  = pd.Series(last_vals[-7:]).std()
        row['roll_std28'] = pd.Series(last_vals[-28:]).std()

        # 시퀀스 구성
        seq_df = history.tail(seq_len).copy()
        seq_scaled = scaler_X.transform(seq_df[feature_cols])
        X = torch.tensor(seq_scaled, dtype=torch.float32).unsqueeze(0)

        # 예측
        pred_scaled = model(X).item()
        pred = scaler_y.inverse_transform([[pred_scaled]])[0][0]
        pred = max(pred, 0)

        preds.append(pred)
        row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([row])], ignore_index=True)

    return preds

# =========================================================
# 🔥 메인 실행
# =========================================================
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자'] = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)

# 학사일정 merge
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')
academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend':'acad_weekend',
    'holiday':'acad_holiday',
    'semester':'acad_semester',
    'seasonal':'acad_seasonal',
    'exam':'acad_exam',
    'ceremony':'acad_ceremony',
    'dormitory':'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test = test.merge(academic, on='영업일자', how='left')

train = add_date_features(train)
test = add_date_features(test)

train = add_lag_features(train).dropna().reset_index(drop=True)
test = add_lag_features(test)

# =========================================================
# 최종 Feature 15개
# =========================================================
feature_cols = [
    'acad_weekend', 'acad_semester', 'acad_weekday',
    'open_hours', 'acad_ceremony', 'acad_exam',
    'lag1','lag7','lag14','lag28',
    'roll_std7','roll_std28',
    'roll_mean7','roll_mean14','roll_mean28'
]

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[['일매출']] = scaler_y.fit_transform(train[['일매출']])

# =========================================================
# Sequence Length = 28
# =========================================================
seq_len = 28
X_train, y_train = create_sequences(train, feature_cols, seq_len)

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# ⭐ 2-Layer GRU 모델
model = GRUModel(input_dim=len(feature_cols), num_layers=2)

criterion = nn.MSELoss()

# ⭐ Adam → SGD(m=0.9)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 학습 50 epochs
for epoch in range(70):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        loss = criterion(model(Xb), yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/70] Loss: {total_loss:.4f}")

# =========================================================
# Test 예측
# =========================================================
test_pred = autoregressive_forecast(model, train, test, seq_len, feature_cols, scaler_X, scaler_y)

mae, rmse, smape_val = evaluate(test['일매출'], test_pred)

print("\n====== 최종 성능 (2-Layer GRU + 15 features + seq_len=28) ======")
print("MAE :", mae)
print("RMSE:", rmse)
print("SMAPE:", smape_val)

test['예측매출'] = test_pred
print(test[['영업일자', '예측매출']])


[Epoch 1/70] Loss: 1.4417
[Epoch 2/70] Loss: 0.7957
[Epoch 3/70] Loss: 0.7591
[Epoch 4/70] Loss: 0.7328
[Epoch 5/70] Loss: 0.7144
[Epoch 6/70] Loss: 0.7033
[Epoch 7/70] Loss: 0.6885
[Epoch 8/70] Loss: 0.6853
[Epoch 9/70] Loss: 0.6709
[Epoch 10/70] Loss: 0.6628
[Epoch 11/70] Loss: 0.6571
[Epoch 12/70] Loss: 0.6457
[Epoch 13/70] Loss: 0.6435
[Epoch 14/70] Loss: 0.6401
[Epoch 15/70] Loss: 0.6268
[Epoch 16/70] Loss: 0.6258
[Epoch 17/70] Loss: 0.6158
[Epoch 18/70] Loss: 0.6135
[Epoch 19/70] Loss: 0.6080
[Epoch 20/70] Loss: 0.5912
[Epoch 21/70] Loss: 0.5841
[Epoch 22/70] Loss: 0.5867
[Epoch 23/70] Loss: 0.5823
[Epoch 24/70] Loss: 0.5796
[Epoch 25/70] Loss: 0.5725
[Epoch 26/70] Loss: 0.5676
[Epoch 27/70] Loss: 0.5763
[Epoch 28/70] Loss: 0.5706
[Epoch 29/70] Loss: 0.5628
[Epoch 30/70] Loss: 0.5652
[Epoch 31/70] Loss: 0.5746
[Epoch 32/70] Loss: 0.5586
[Epoch 33/70] Loss: 0.5651
[Epoch 34/70] Loss: 0.5490
[Epoch 35/70] Loss: 0.5527
[Epoch 36/70] Loss: 0.5599
[Epoch 37/70] Loss: 0.5525
[Epoch 38/

**Adagrad**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# =========================================================
# 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v

# =========================================================
# 데이터 전처리 함수
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출']
        .astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df

def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    df['roll_std28'] = df['일매출'].rolling(28).std()
    return df

# =========================================================
# Sliding Window
# =========================================================
def create_sequences(df, feature_cols, seq_len=28):
    X, y = [], []
    values = df[feature_cols].values
    targets = df['일매출'].values
    for i in range(seq_len, len(df)):
        X.append(values[i-seq_len:i])
        y.append(targets[i])
    return np.array(X), np.array(y)

# =========================================================
# Dataset
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# =========================================================
# ⭐⭐ 2-Layer GRU 모델 ⭐⭐
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.gru = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])

# =========================================================
# Autoregressive 예측
# =========================================================
def autoregressive_forecast(model, df_train, df_test, seq_len, feature_cols, scaler_X, scaler_y):

    history = df_train.copy()
    preds = []

    for i in range(len(df_test)):
        row = df_test.iloc[i].copy()
        last_vals = history['일매출'].values

        # lag 업데이트
        row['lag1'] = last_vals[-1]
        row['lag7'] = last_vals[-7]
        row['lag14'] = last_vals[-14]
        row['lag28'] = last_vals[-28]

        # rolling 업데이트
        row['roll_mean7']  = pd.Series(last_vals[-7:]).mean()
        row['roll_mean14'] = pd.Series(last_vals[-14:]).mean()
        row['roll_mean28'] = pd.Series(last_vals[-28:]).mean()

        row['roll_std7']  = pd.Series(last_vals[-7:]).std()
        row['roll_std28'] = pd.Series(last_vals[-28:]).std()

        # 시퀀스 구성
        seq_df = history.tail(seq_len).copy()
        seq_scaled = scaler_X.transform(seq_df[feature_cols])
        X = torch.tensor(seq_scaled, dtype=torch.float32).unsqueeze(0)

        # 예측
        pred_scaled = model(X).item()
        pred = scaler_y.inverse_transform([[pred_scaled]])[0][0]
        pred = max(pred, 0)

        preds.append(pred)
        row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([row])], ignore_index=True)

    return preds

# =========================================================
# 🔥 메인 실행
# =========================================================
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자'] = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)

# 학사일정 merge
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')
academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend':'acad_weekend',
    'holiday':'acad_holiday',
    'semester':'acad_semester',
    'seasonal':'acad_seasonal',
    'exam':'acad_exam',
    'ceremony':'acad_ceremony',
    'dormitory':'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test = test.merge(academic, on='영업일자', how='left')

train = add_date_features(train)
test = add_date_features(test)

train = add_lag_features(train).dropna().reset_index(drop=True)
test = add_lag_features(test)

# =========================================================
# 최종 Feature 15개
# =========================================================
feature_cols = [
    'acad_weekend', 'acad_semester', 'acad_weekday',
    'open_hours', 'acad_ceremony', 'acad_exam',
    'lag1','lag7','lag14','lag28',
    'roll_std7','roll_std28',
    'roll_mean7','roll_mean14','roll_mean28'
]

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[['일매출']] = scaler_y.fit_transform(train[['일매출']])

# =========================================================
# Sequence Length = 28
# =========================================================
seq_len = 28
X_train, y_train = create_sequences(train, feature_cols, seq_len)

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# 2-Layer GRU 모델
model = GRUModel(input_dim=len(feature_cols), num_layers=2)

criterion = nn.MSELoss()

# Adagrad
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.001)

# 학습 50 epochs
for epoch in range(50):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        loss = criterion(model(Xb), yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/50] Loss: {total_loss:.4f}")


# =========================================================
# Test 예측
# =========================================================
test_pred = autoregressive_forecast(model, train, test, seq_len, feature_cols, scaler_X, scaler_y)

mae, rmse, smape_val = evaluate(test['일매출'], test_pred)

print("\n====== 최종 성능 (2-Layer GRU + 15 features + seq_len=28) ======")
print("MAE :", mae)
print("RMSE:", rmse)
print("SMAPE:", smape_val)

test['예측매출'] = test_pred
print(test[['영업일자', '예측매출']])


[Epoch 1/50] Loss: 0.8402
[Epoch 2/50] Loss: 0.6291
[Epoch 3/50] Loss: 0.5809
[Epoch 4/50] Loss: 0.5459
[Epoch 5/50] Loss: 0.5305
[Epoch 6/50] Loss: 0.5117
[Epoch 7/50] Loss: 0.4972
[Epoch 8/50] Loss: 0.4938
[Epoch 9/50] Loss: 0.4787
[Epoch 10/50] Loss: 0.4707
[Epoch 11/50] Loss: 0.4673
[Epoch 12/50] Loss: 0.4472
[Epoch 13/50] Loss: 0.4424
[Epoch 14/50] Loss: 0.4389
[Epoch 15/50] Loss: 0.4242
[Epoch 16/50] Loss: 0.4227
[Epoch 17/50] Loss: 0.4116
[Epoch 18/50] Loss: 0.4053
[Epoch 19/50] Loss: 0.3967
[Epoch 20/50] Loss: 0.3838
[Epoch 21/50] Loss: 0.3793
[Epoch 22/50] Loss: 0.3837
[Epoch 23/50] Loss: 0.3730
[Epoch 24/50] Loss: 0.3713
[Epoch 25/50] Loss: 0.3665
[Epoch 26/50] Loss: 0.3650
[Epoch 27/50] Loss: 0.3649
[Epoch 28/50] Loss: 0.3641
[Epoch 29/50] Loss: 0.3584
[Epoch 30/50] Loss: 0.3604
[Epoch 31/50] Loss: 0.3627
[Epoch 32/50] Loss: 0.3516
[Epoch 33/50] Loss: 0.3550
[Epoch 34/50] Loss: 0.3492
[Epoch 35/50] Loss: 0.3520
[Epoch 36/50] Loss: 0.3543
[Epoch 37/50] Loss: 0.3469
[Epoch 38/